In [1]:
import numpy as np 
import torch
from scipy import signal
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from collections import Counter
from scipy.spatial import KDTree

In [2]:
num_adc_bits = 16
num_adc_samples = 128
num_rx = 4
num_tx = 3
num_lanes = 2 
is_real = 0
num_chirps_in_frame = 256
f0 = 60      # min frequency
B = 3239     # sweep bandwidth
S = 54       # sweep slope
fs = 4000    # sampling freq
Nc = 256     # num chirps in 1 frame
Ns = 128     # num samples of 1 chirp 
c0 = 3e+8
idle_time = 5e-6

fc = f0+B/2000   
lamda = c0/(fc*1e+9)
Tc= idle_time + (B/S)*(1e-6)

R_res = c0 / (2e+6*B)       
R_max = fs*c0 / (2e+9*S)
velocity_res = lamda/(2*Nc*Tc)
v_max = lamda/(4*Tc)

print(R_res)
print(R_max)
print(velocity_res)
print(v_max)

0.04631058968817536
11.11111111111111
0.14633341751423926
18.730677441822625


In [3]:
def adc_processing(adc_data):
    if num_adc_bits != 16: 
        l_max = 2**(num_adc_bits-1) -1 
        adc_data[adc_data>l_max] -= 2**num_adc_bits 

    file_size = adc_data.size
    num_chirps = file_size // (2 * num_adc_samples * num_rx)
    file_size = num_chirps * (2 * num_adc_samples * num_rx)
    adjusted_size = (file_size//4) * 4

    lvds = np.zeros((file_size//2), dtype = complex)
    lvds[0::2] = adc_data[0:adjusted_size:4] + 1j*adc_data[2:adjusted_size:4]
    lvds[1::2] = adc_data[1:adjusted_size:4] + 1j*adc_data[3:adjusted_size:4]
    lvds = lvds.reshape((num_chirps), num_adc_samples*num_rx)

    new_adc_data = np.zeros((num_rx, num_chirps * num_adc_samples), dtype = np.complex128)
    for row in range(num_rx):
        for i in range(num_chirps):
            new_adc_data[row, i*num_adc_samples:(i+1)*num_adc_samples] = lvds[i, row*num_adc_samples:(row+1)*num_adc_samples]
    
    rx1 = np.reshape(new_adc_data[0,:],(num_chirps,128))
    rx2 = np.reshape(new_adc_data[1,:],(num_chirps,128)) 
    rx3 = np.reshape(new_adc_data[2,:],(num_chirps,128))
    rx4 = np.reshape(new_adc_data[3,:],(num_chirps,128))

    rx1_tx1 = rx1[0::3]
    rx1_tx2 = rx1[1::3]
    rx1_tx3 = rx1[2::3]

    num_frames = num_chirps // (num_chirps_in_frame*num_tx)
    data_rx1_tx1 = rx1_tx1[0:num_frames*num_chirps_in_frame]
    data_rx1_tx1 = data_rx1_tx1.reshape((num_frames,num_chirps_in_frame,num_adc_samples))

    data_rx1_tx2 = rx1_tx2[0:num_frames*num_chirps_in_frame]
    data_rx1_tx2 = data_rx1_tx2.reshape((num_frames,num_chirps_in_frame,num_adc_samples))

    data_rx1_tx3 = rx1_tx3[0:num_frames*num_chirps_in_frame]
    data_rx1_tx3 = data_rx1_tx3.reshape((num_frames,num_chirps_in_frame,num_adc_samples))

    rx2_tx1 = rx2[0::3]
    rx2_tx2 = rx2[1::3]
    rx2_tx3 = rx2[2::3]

    data_rx2_tx1 = rx2_tx1[0:num_frames*num_chirps_in_frame]
    data_rx2_tx1 = data_rx2_tx1.reshape((num_frames,num_chirps_in_frame,num_adc_samples))
    
    data_rx2_tx2 = rx2_tx2[0:num_frames*num_chirps_in_frame]
    data_rx2_tx2 = data_rx2_tx2.reshape((num_frames,num_chirps_in_frame,num_adc_samples))

    data_rx2_tx3 = rx2_tx3[0:num_frames*num_chirps_in_frame]
    data_rx2_tx3 = data_rx2_tx3.reshape((num_frames,num_chirps_in_frame,num_adc_samples))

    rx3_tx1 = rx3[0::3]
    rx3_tx2 = rx3[1::3]
    rx3_tx3 = rx3[2::3]

    data_rx3_tx1 = rx3_tx1[0:num_frames*num_chirps_in_frame]
    data_rx3_tx1 = data_rx3_tx1.reshape((num_frames,num_chirps_in_frame,num_adc_samples))
    
    data_rx3_tx2 = rx3_tx2[0:num_frames*num_chirps_in_frame]
    data_rx3_tx2 = data_rx3_tx2.reshape((num_frames,num_chirps_in_frame,num_adc_samples))

    data_rx3_tx3 = rx3_tx3[0:num_frames*num_chirps_in_frame]
    data_rx3_tx3 = data_rx3_tx3.reshape((num_frames,num_chirps_in_frame,num_adc_samples))

    rx4_tx1 = rx4[0::3]
    rx4_tx2 = rx4[1::3]
    rx4_tx3 = rx4[2::3]

    data_rx4_tx1 = rx4_tx1[0:num_frames*num_chirps_in_frame]
    data_rx4_tx1 = data_rx4_tx1.reshape((num_frames,num_chirps_in_frame,num_adc_samples))
    
    data_rx4_tx2 = rx4_tx2[0:num_frames*num_chirps_in_frame]
    data_rx4_tx2 = data_rx4_tx2.reshape((num_frames,num_chirps_in_frame,num_adc_samples))
    
    data_rx4_tx3 = rx4_tx3[0:num_frames*num_chirps_in_frame]
    data_rx4_tx3 = data_rx4_tx3.reshape((num_frames,num_chirps_in_frame,num_adc_samples))


    print(num_frames)
    return data_rx1_tx1 , data_rx1_tx2, data_rx1_tx3, data_rx2_tx1, data_rx2_tx2, data_rx2_tx3, data_rx3_tx1, data_rx3_tx2, data_rx3_tx3, data_rx4_tx1, data_rx4_tx2, data_rx4_tx3


In [4]:
file_name = "Nguyen_di_ngang_3m_2_ban.bin"
with open(file_name,'rb') as fid:
    adc_data = np.fromfile(fid, dtype = np.int16)

data_rx1_tx1 , data_rx1_tx2, data_rx1_tx3, data_rx2_tx1, data_rx2_tx2, data_rx2_tx3, data_rx3_tx1, data_rx3_tx2, data_rx3_tx3, data_rx4_tx1, data_rx4_tx2, data_rx4_tx3 = adc_processing(adc_data)

288


In [5]:
def doppler_fft_without_butter(data): 
    data_time = data
    tmp = np.fft.fft(data_time, axis = 0)
    doppler_fft = np.fft.fft(tmp, axis = 1)
    return doppler_fft

In [6]:
def doppler_fft(data): 
    data_time = data
    tmp = np.fft.fft(data_time, axis = 0)

    b, a = signal.butter(4, 0.0075, 'high')
    data_range = signal.lfilter(b, a, tmp, axis = 1)

    doppler_fft = np.fft.fft(data_range, axis = 1)
    return doppler_fft

In [7]:
def update_column(matrix, N):
    # Xử lý việc cập nhật cột thứ N
    M = len(matrix)  # kích thước của ma trận MxM
    
    # Duyệt qua từng phần tử của cột N (trừ các phần tử đầu và cuối vì không có đủ phần tử bên trái hoặc phải)
    for i in range(M):
        # Tính giá trị trung bình cộng của phần tử bên trái và bên phải
        matrix[i][N] = (matrix[i][N - 1] + matrix[i][N + 1]) / 2
    
    return matrix

In [8]:
def getNeighbors(P, D, eps):
  """
  Tìm các điểm lân cận của điểm P trong tập dữ liệu D với bán kính eps.

  Args:
    P: Điểm cần tìm lân cận.
    D: Tập dữ liệu.
    eps: Bán kính lân cận.

  Returns:
    Danh sách các điểm lân cận.
  """
  # Scale lại tọa độ y của D (chỉ scale cột y)
  D_scaled = D.copy()
  D_scaled[:, 1] = D[:, 1] / 3  
  
  # Scale lại tọa độ y của P (chỉ scale phần tử thứ 2)
  P_scaled = P.copy()
  P_scaled[1] = P[1] / 3

  tree = KDTree(D_scaled[:, :2])
  indices = tree.query_ball_point(P_scaled[:2], eps)

  # Lọc kết quả bằng indexing
  neighbors = D[indices][D[indices, 2] <= P[2]] 
  
  # Chuyển đổi kết quả sang list
  return neighbors.tolist() 

def tim_hang(D, X):
  """
  Kiểm tra sự tồn tại của hàng trong ma trận D có giá trị bằng với mảng X.

  Args:
    D: Ma trận NumPy.
    X: Mảng NumPy.

  Returns:
    True nếu tìm thấy hàng trong ma trận D, ngược lại trả về False.
  """
  return np.any(np.all(D == X, axis=1))

def xoa_hang(D, X):
  """
  Xóa hàng trong ma trận D có giá trị bằng với mảng X (tối ưu).

  Args:
    D: Ma trận NumPy.
    X: Mảng NumPy.

  Returns:
    Ma trận NumPy mới sau khi xóa hàng.
  """
  mask = ~np.all(D == X, axis=1)
  return D[mask]

def get_rows_not_in_set(D, C):
  """
  Lấy các hàng trong ma trận D mà không xuất hiện trong set C.

  Args:
    D: Ma trận đầu vào.
    C: Set đầu vào.

  Returns:
    Ma trận chứa các hàng trong D nhưng không trong C.
  """
  # Chuyển đổi set C thành mảng numpy
  C_np = np.array(list(C))
  # Tìm các hàng trong D không có trong C
  mask = ~np.any(np.all(D[:, np.newaxis] == C_np, axis=2), axis=1)
  return D[mask]

def KBSCAN(D,eps,MinPts):
    clusters = {}
    C = 1
    unvisited = D
    cluster_points = set()

    while(unvisited.size != 0):
        index_max = np.where(unvisited[:,2] == np.max(unvisited[:,2]))
        P = np.array(unvisited[index_max]).reshape(3,)

        neighbors = getNeighbors(P, D, eps)
        if tim_hang(unvisited,P):
            unvisited = xoa_hang(unvisited,P)
        if len(neighbors) >= MinPts: 
            clusters[C] = []
            clusters[C].append(P)
            cluster_points.add(tuple(P))
            for P_prime in neighbors:
                if tim_hang(unvisited,P_prime):
                    unvisited = xoa_hang(unvisited,P_prime)
                    neighbors_prime = getNeighbors(P_prime,D,eps)
                    if len(neighbors_prime) >= MinPts:
                        neighbors.extend(neighbors_prime)
                if tuple(P_prime) not in cluster_points:
                    clusters[C].append(P_prime)
                    cluster_points.add(tuple(P_prime))
            C += 1

    clusters[-1] = []
    clusters[-1].extend(get_rows_not_in_set(D, np.array(list(cluster_points))))

    Y = np.zeros((128,256))
    max_elements = 0
    index_max_elements = None
    for i in clusters:
        if len(clusters[i]) > max_elements:
            max_elements = len(clusters[i])
            index_max_elements = i

    # Lấy tọa độ x, y của các điểm trong cluster
    x_coords = np.array([int(row[0]) for row in clusters[index_max_elements]])
    y_coords = np.array([int(row[1]) for row in clusters[index_max_elements]])
    # Gán giá trị i cho các tọa độ tương ứng trong ma trận Y
    Y[x_coords, y_coords] = 1
    return Y

In [9]:
def KBSCAN_plot(data):
    #Data preprocessing
    data_test = doppler_fft_without_butter(data.T)
    doppler_magnitude = np.abs(data_test)
    velocity_bins = np.fft.fftfreq(Nc, d=1/Nc) * velocity_res
    doppler_magnitude = np.fft.fftshift(doppler_magnitude, axes=1)
    velocity_bins = np.fft.fftshift(velocity_bins)

    output_test = 20 * np.log10(doppler_magnitude)
    output_test = update_column(output_test,128)

    output_test_2 = np.copy(output_test)
    output_test_2[output_test > 70] = 100
    output_test_2[output_test <= 70] = 0

    indices = np.column_stack(np.where(output_test_2 == 100))

    #DBSCAN
    db = DBSCAN(eps=3, min_samples=3)
    labels = db.fit_predict(indices)
    label_counts = Counter(labels)
    most_frequent_label = label_counts.most_common(1)[0][0]
    new_labels = [1 if label == most_frequent_label else 0 for label in labels]
    Y = np.zeros_like(output_test)
    for i, label in enumerate(new_labels):
        if label != -1:  # -1 là chỉ các điểm nhiễu (outliers) trong DBSCAN
            Y[indices[i, 0], indices[i, 1]] = label  # Gán nhãn bắt đầu từ 1
    indices_2 = np.column_stack(np.where(Y == 1))
    out = np.copy(output_test)
    out[Y < 0.5] = 0
    k = (out[out>0]).reshape(-1,1)

    #KBSCAN
    D = np.concatenate((indices_2,k),axis=1)
    Y = KBSCAN(D,2,3)
    
    return Y

In [10]:
# Hàm plot_final nhận từng data và trả về output_test
def plot_final_full(data):
    data_test = doppler_fft_without_butter(data.T)
    doppler_magnitude = np.abs(data_test)
    velocity_bins = np.fft.fftfreq(Nc, d=1 / Nc) * velocity_res

    # Shift zero-frequency component to center of spectrum
    doppler_magnitude = np.fft.fftshift(doppler_magnitude, axes=1)
    velocity_bins = np.fft.fftshift(velocity_bins)

    output_test = 20 * np.log10(doppler_magnitude)
    output_test = update_column(output_test, 128)

    output_test_2 = np.copy(output_test)
    output_test_2[output_test > 70] = 100
    output_test_2[output_test <= 70] = 0

    indices = np.column_stack(np.where(output_test_2 == 100))
    db = DBSCAN(eps=3, min_samples=3)
    labels = db.fit_predict(indices)

    label_counts = Counter(labels)

    # Tìm cụm có số phần tử nhiều nhất
    most_frequent_label = label_counts.most_common(1)[0][0]

    # Gán nhãn mới
    new_labels = [1 if label == most_frequent_label else 0 for label in labels]

    # Gán nhãn cho ma trận Y (cùng kích thước với ma trận X)
    Y = np.zeros_like(output_test)
    for i, label in enumerate(new_labels):
        if label != -1:  # -1 là chỉ các điểm nhiễu (outliers) trong DBSCAN
            Y[indices[i, 0], indices[i, 1]] = label  # Gán nhãn bắt đầu từ 1

    mean = np.mean(output_test)
    output_test[Y < 0.5] = mean

    nhan = KBSCAN_plot(data)
    output_test = output_test * nhan

    return output_test

# Danh sách data đầu vào từ adc_processing
(
    data_rx1_tx1, data_rx1_tx2, data_rx1_tx3,
    data_rx2_tx1, data_rx2_tx2, data_rx2_tx3,
    data_rx3_tx1, data_rx3_tx2, data_rx3_tx3,
    data_rx4_tx1, data_rx4_tx2, data_rx4_tx3
) = adc_processing(adc_data)

f = 1 # Chỉ số frame cần xử lý
# Tạo danh sách các data cần xử lý
data_list = [
    data_rx1_tx1[f], data_rx1_tx2[f], data_rx1_tx3[f],
    data_rx2_tx1[f], data_rx2_tx2[f], data_rx2_tx3[f],
    data_rx3_tx1[f], data_rx3_tx2[f], data_rx3_tx3[f],
    data_rx4_tx1[f], data_rx4_tx2[f], data_rx4_tx3[f]
]

data_list_full = [
    data_rx1_tx1, data_rx1_tx2, data_rx1_tx3,
    data_rx2_tx1, data_rx2_tx2, data_rx2_tx3,
    data_rx3_tx1, data_rx3_tx2, data_rx3_tx3,
    data_rx4_tx1, data_rx4_tx2, data_rx4_tx3
]

# Xử lý tất cả các data và lưu kết quả
output_tests = [plot_final_full(data) for data in data_list] # data_list_full

# output_tests chứa kết quả output_test cho từng data


288


In [11]:
def inverse_doppler_fft(ifft_data):
    # Thực hiện inverse FFT trên trục 1 trước
    tmp = np.fft.ifft(ifft_data, axis=1)
    # Sau đó thực hiện inverse FFT trên trục 0
    data_time = np.fft.ifft(tmp, axis=0)
    return data_time

In [12]:
# Định nghĩa dải frame cần xử lý
f_start = 0  # Bắt đầu từ frame 0
f_end = 288  # Kết thúc tại frame 287 (tổng cộng 288 frames)

# Tạo danh sách để lưu kết quả
output_tests_rx1_tx1 = []

# Duyệt qua từng frame và xử lý
for f in range(f_start, f_end):
    # Chọn dữ liệu frame f của data_rx1_tx1
    data = data_rx1_tx1[f]
    
    # Áp dụng hàm plot_final_full cho dữ liệu frame hiện tại
    output_test = plot_final_full(data)
    output_test = 10 ** (output_test / 20)   
    output_test = inverse_doppler_fft(output_test).T
    
    # Lưu kết quả của frame vào danh sách
    output_tests_rx1_tx1.append(output_test)

# Sau khi hoàn thành, output_tests_rx1_tx1 chứa kết quả của 288 frame.

# Tạo danh sách để lưu kết quả
output_tests_rx1_tx2 = []

# Duyệt qua từng frame và xử lý
for f in range(f_start, f_end):
    # Chọn dữ liệu frame f của data_rx1_tx1
    data = data_rx1_tx2[f]
    
    # Áp dụng hàm plot_final_full cho dữ liệu frame hiện tại
    output_test = plot_final_full(data)
    output_test = 10 ** (output_test / 20)   

    output_test = inverse_doppler_fft(output_test).T
    
    # Lưu kết quả của frame vào danh sách
    output_tests_rx1_tx2.append(output_test)


# Tạo danh sách để lưu kết quả
output_tests_rx1_tx3 = []

# Duyệt qua từng frame và xử lý
for f in range(f_start, f_end):
    # Chọn dữ liệu frame f của data_rx1_tx1
    data = data_rx1_tx3[f]
    
    # Áp dụng hàm plot_final_full cho dữ liệu frame hiện tại
    output_test = plot_final_full(data)
    output_test = 10 ** (output_test / 20)   
    output_test = inverse_doppler_fft(output_test).T
    # Lưu kết quả của frame vào danh sách
    output_tests_rx1_tx3.append(output_test)




In [13]:
### vẽ

#velocity_bins = np.fft.fftfreq(Nc, d=1/Nc) * velocity_res
# velocity_bins = np.fft.fftshift(velocity_bins)
# # Plot the Range-Velocity Map
# plt.figure(figsize=(30, 12))
# im =plt.imshow(output_tests[0], aspect='auto', cmap='jet', extent=[velocity_bins.min(), velocity_bins.max(), R_max, 0])
# # Set the colorbar limits
# clim = im.get_clim()
# im.set_clim(clim[1]- 40, clim[1])
# plt.title('Range-Velocity Map')
# plt.ylabel('Range (m)')
# plt.xlabel('Velocity (m/s)')
# plt.colorbar(label='Magnitude (dB)')
# plt.show()

In [14]:
output_tests_rx1_tx1 = np.array(output_tests_rx1_tx1)
output_tests_rx1_tx2 = np.array(output_tests_rx1_tx2)
output_tests_rx1_tx3 = np.array(output_tests_rx1_tx3)

In [15]:
output_tests_rx1_tx1.shape

(288, 256, 128)

In [16]:
def reshape_output_tests(output_tests):
    return output_tests.transpose(2, 0, 1).reshape(128, -1)

# Chuyển đổi tất cả output_tests
rD1_1 = reshape_output_tests(output_tests_rx1_tx1)
rD1_2 = reshape_output_tests(output_tests_rx1_tx2)
rD1_3 = reshape_output_tests(output_tests_rx1_tx3)


In [17]:
rD1_1.shape

(128, 73728)

In [18]:
# Số cột trong mỗi mảng
num_columns = rD1_1.shape[1]

# Tạo danh sách để lưu các cột đã hoán đổi
reordered_columns = []

# Duyệt qua từng cột
for i in range(num_columns):
    reordered_columns.append(rD1_1[:, i])  # Cột thứ i của rD1_1
    reordered_columns.append(rD1_2[:, i])  # Cột thứ i của rD1_2
    reordered_columns.append(rD1_3[:, i])  # Cột thứ i của rD1_3

# Chuyển danh sách thành mảng numpy
result = np.column_stack(reordered_columns)

# Kiểm tra kích thước kết quả
print("Kích thước mảng kết quả:", result.shape)


Kích thước mảng kết quả: (128, 221184)


In [22]:
Data_time = rD1_1
tmp = np.fft.fft(Data_time, axis=0)
b, a = signal.butter(4, 0.0075, 'high')   
# Perform Range
Data_range_MTI = signal.lfilter(b,a,tmp, axis=1)
rD1 = Data_range_MTI

In [ ]:
mD_max = 865 # xem code đức check 
n_chirps = 221184 # ngắn hơn do bị cắt lúc đầu

fig, ax = plt.subplots(figsize=(30, 12))
# Plot the spectrogram

#epsilon = 1e-10  # Giá trị nhỏ để tránh log(0)
im = ax.imshow(20 * np.log10(np.abs(rD1)), cmap='jet', aspect='auto')
# Set the axis labels and title
ax.set_xlabel('No. of Sweeps')
ax.set_ylabel('Range')
ax.set_title('Range Profiles after MTI filter')

#Set the y-axis ticks and labels
yticks = np.linspace(0, 127, 10, dtype=int)
ax.set_yticks(yticks)
ax.set_yticklabels(['{:.2f}'.format(i*R_max/127) for i in yticks])

xticks = np.linspace(0, mD_max, 30, dtype=int)
ax.set_xticks(xticks)
ax.set_xticklabels(['{:.0f}'.format(i*n_chirps/mD_max) for i in xticks])

# Set the colorbar limits
clim = im.get_clim()
im.set_clim(clim[0] + 20 , clim[1])

# Flip the y-axis to match MATLAB's behavior
ax.invert_yaxis()

# # Adjust the y-axis limits
# ax.set_ylim([0, 127])

# Add the colorbar
cbar = fig.colorbar(im, ax=ax)
cbar.set_label('Magnitude (dB)')

# Show the plot
plt.show()